In [103]:
import requests

# Obtén la lista de todos los pares de criptomonedas y sus detalles
base_url_exchange_info = 'https://api.binance.com/api/v3/exchangeInfo'
response_exchange_info = requests.get(base_url_exchange_info)
exchange_info = response_exchange_info.json()

# Filtra los pares de criptomonedas contra USDT
usdt_pairs = [pair['symbol'] for pair in exchange_info['symbols'] if pair['quoteAsset'] == 'USDT']

# Obtén los precios y volúmenes de las criptomonedas
base_url_ticker = 'https://api.binance.com/api/v3/ticker/24hr'
response_ticker = requests.get(base_url_ticker)
ticker_data = response_ticker.json()

# Calcula el market cap para cada criptomoneda y crea una lista de tuplas (symbol, market cap)
market_cap_data = [(data['symbol'], float(data['weightedAvgPrice']) * float(data['quoteVolume'])) for data in ticker_data if data['symbol'] in usdt_pairs]

# Ordena la lista de tuplas según el market cap de mayor a menor
sorted_market_cap_data = sorted(market_cap_data, key=lambda x: x[1], reverse=True)

# Imprime las mejores criptomonedas según su market cap
for i, (symbol, market_cap) in enumerate(sorted_market_cap_data[:10], start=1):
    print(f"{i}. Symbol: {symbol}, Market Cap: {market_cap:.2f}")


1. Symbol: BTCUSDT, Market Cap: 14841335015356.29
2. Symbol: ETHUSDT, Market Cap: 517162207639.19
3. Symbol: BNBUSDT, Market Cap: 26987208853.43
4. Symbol: MKRUSDT, Market Cap: 10280131523.48
5. Symbol: WBTCUSDT, Market Cap: 6847197620.40
6. Symbol: YFIUSDT, Market Cap: 3702653976.05
7. Symbol: LTCUSDT, Market Cap: 2200147129.01
8. Symbol: BCHUSDT, Market Cap: 1973485091.45
9. Symbol: PAXGUSDT, Market Cap: 1912843593.44
10. Symbol: XMRUSDT, Market Cap: 1231366307.01


Market cap = circulating supply x current price<br>
you can get the circulation supply (cs) of a pair from binance:
exchange-api/v1/public/asset-service/product/get-products

In [105]:
import requests

# Endpoint para obtener la lista de criptomonedas
base_url = 'https://api.binance.com/api/v3/ticker/24hr'

# Realiza la solicitud GET a la URL
response = requests.get(base_url)
data = response.json()

# Filtra y ordena las criptomonedas por volumen en las últimas 24 horas
top_crypto_symbols = sorted(data, key=lambda x: float(x['volume']), reverse=True)[:10]

# Imprime los datos de las 10 criptomonedas con mayor volumen
for i, crypto_data in enumerate(top_crypto_symbols, start=1):
    print(f"{i}. Symbol: {crypto_data['symbol']}, Volume: {crypto_data['volume']}")


1. Symbol: PEPEUSDT, Volume: 8497146306355.00
2. Symbol: SHIBUSDT, Volume: 2823116184106.00
3. Symbol: PEPETRY, Volume: 2814480065178.00
4. Symbol: BTTCUSDT, Volume: 817460614889.0
5. Symbol: SHIBBUSD, Volume: 405841488334.00
6. Symbol: SHIBTRY, Volume: 327493997761.00
7. Symbol: BTTCBUSD, Volume: 203257991436.0
8. Symbol: BTTCTRY, Volume: 126469691740.0
9. Symbol: FLOKIUSDT, Volume: 60176428367.00
10. Symbol: LUNCUSDT, Volume: 39321811383.24000000


In [110]:
import requests
import datetime

# URL para obtener la lista de pares de criptomonedas en Binance
base_url = 'https://api.binance.com/api/v3/exchangeInfo'

# Obtiene la lista de todos los pares de criptomonedas
response = requests.get(base_url)
data = response.json()

# Filtra los pares de criptomonedas para quedarte solo con aquellos que son contra USDT
usdt_pairs = [pair for pair in data['symbols'] if pair['quoteAsset'] == 'USDT']

# Define el rango de fechas para el análisis (desde el 1 de enero hasta el 31 de diciembre)
start_date = int(datetime.datetime(2023, 1, 1).timestamp() * 1000)
end_date = int(datetime.datetime(2023, 12, 31).timestamp() * 1000)

# Lista para almacenar los datos de las criptomonedas
crypto_data = []

# Recorre los pares de criptomonedas contra USDT
for pair in usdt_pairs:
    symbol = pair['symbol']
    
    # Obtiene los datos de OHLCV para el par de criptomonedas en el rango de fechas
    ohlcv_url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_date}&endTime={end_date}'
    ohlcv_response = requests.get(ohlcv_url)
    ohlcv_data = ohlcv_response.json()
    
    # Verifica si hay datos de OHLCV disponibles
    if len(ohlcv_data) < 2:
        continue
    
    # Calcula el rendimiento
    start_price = float(ohlcv_data[0][1])
    end_price = float(ohlcv_data[-1][4])
    percentage_change = ((end_price - start_price) / start_price) * 100
    
    # Obtiene los datos actuales del par de criptomonedas
    ticker_url = f'https://api.binance.com/api/v3/ticker/24hr?symbol={symbol}'
    ticker_response = requests.get(ticker_url)
    ticker_data = ticker_response.json()
    
    # Calcula la capitalización de mercado estimada
    market_cap = float(ticker_data['quoteVolume']) * float(ticker_data['lastPrice'])
    
    # Obtiene el volumen de trading
    volume = float(ticker_data['quoteVolume'])
    
    # Agrega los datos al lista de crypto_data
    crypto_data.append({
        'symbol': symbol,
        'percentage_change': percentage_change,
        'market_cap': market_cap,
        'volume': volume
    })

# Ordena las criptomonedas en función de la combinación de rendimiento, capitalización de mercado y volumen
sorted_crypto_data = sorted(crypto_data, key=lambda x: (x['percentage_change'], x['market_cap'], x['volume']), reverse=True)

# Imprime las 10 mejores criptomonedas
for i, crypto in enumerate(sorted_crypto_data[:10], start=1):
    print(f"{i}. Symbol: {crypto['symbol']}, Rendimiento: {crypto['percentage_change']:.2f}%, Market Cap: {crypto['market_cap']:.2f}, Volumen: {crypto['volume']:.2f}")


1. Symbol: WLDUSDT, Rendimiento: 864.00%, Market Cap: 13495656.70, Volumen: 9333095.92
2. Symbol: EDUUSDT, Rendimiento: 741.86%, Market Cap: 1510019.15, Volumen: 3587340.30
3. Symbol: ARKMUSDT, Rendimiento: 693.40%, Market Cap: 1905992.54, Volumen: 4804619.47
4. Symbol: IDUSDT, Rendimiento: 659.24%, Market Cap: 399043.03, Volumen: 2102328.79
5. Symbol: TOMOUSDT, Rendimiento: 501.34%, Market Cap: 11927125.20, Volumen: 7602221.43
6. Symbol: INJUSDT, Rendimiento: 467.29%, Market Cap: 53051299.24, Volumen: 7334619.00
7. Symbol: CFXUSDT, Rendimiento: 448.87%, Market Cap: 634088.63, Volumen: 5227441.32
8. Symbol: SUIUSDT, Rendimiento: 431.90%, Market Cap: 17079812.17, Volumen: 32110945.98
9. Symbol: MAVUSDT, Rendimiento: 312.80%, Market Cap: 233324.94, Volumen: 1130450.29
10. Symbol: AKROUSDT, Rendimiento: 285.38%, Market Cap: 401228.87, Volumen: 34588695.77


In [111]:
import requests
import datetime

# URL para obtener la lista de pares de criptomonedas en Binance
base_url = 'https://api.binance.com/api/v3/exchangeInfo'

# Obtiene la lista de todos los pares de criptomonedas
response = requests.get(base_url)
data = response.json()

# Filtra los pares de criptomonedas para quedarte solo con aquellos que son contra USDT
usdt_pairs = [pair for pair in data['symbols'] if pair['quoteAsset'] == 'USDT']

# Calcula las fechas de inicio y fin para un período de 3 meses atrás desde la fecha actual
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=90)

# Convierte las fechas en timestamps
start_timestamp = int(start_date.timestamp() * 1000)
end_timestamp = int(end_date.timestamp() * 1000)

# Lista para almacenar los datos de las criptomonedas
crypto_data = []

# Recorre los pares de criptomonedas contra USDT
for pair in usdt_pairs:
    symbol = pair['symbol']
    
    # Obtiene los datos de OHLCV para el par de criptomonedas en el período de 3 meses
    ohlcv_url = f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval=1d&startTime={start_timestamp}&endTime={end_timestamp}'
    ohlcv_response = requests.get(ohlcv_url)
    ohlcv_data = ohlcv_response.json()
    
    # Verifica si hay datos de OHLCV disponibles
    if len(ohlcv_data) < 2:
        continue
    
    # Calcula el rendimiento
    start_price = float(ohlcv_data[0][1])
    end_price = float(ohlcv_data[-1][4])
    percentage_change = ((end_price - start_price) / start_price) * 100
    
    # Obtiene los datos actuales del par de criptomonedas
    ticker_url = f'https://api.binance.com/api/v3/ticker/24hr?symbol={symbol}'
    ticker_response = requests.get(ticker_url)
    ticker_data = ticker_response.json()
    
    # Calcula la capitalización de mercado estimada
    market_cap = float(ticker_data['quoteVolume']) * float(ticker_data['lastPrice'])
    
    # Obtiene el volumen de trading
    volume = float(ticker_data['quoteVolume'])
    
    # Agrega los datos al lista de crypto_data
    crypto_data.append({
        'symbol': symbol,
        'percentage_change': percentage_change,
        'market_cap': market_cap,
        'volume': volume
    })

# Ordena las criptomonedas en función de la combinación de rendimiento, capitalización de mercado y volumen
sorted_crypto_data = sorted(crypto_data, key=lambda x: (x['percentage_change'], x['market_cap'], x['volume']), reverse=True)

# Imprime las 10 mejores criptomonedas
for i, crypto in enumerate(sorted_crypto_data[:10], start=1):
    print(f"{i}. Symbol: {crypto['symbol']}, Rendimiento: {crypto['percentage_change']:.2f}%, Market Cap: {crypto['market_cap']:.2f}, Volumen: {crypto['volume']:.2f}")


1. Symbol: WLDUSDT, Rendimiento: 859.33%, Market Cap: 13605698.60, Volumen: 9454967.76
2. Symbol: ARKMUSDT, Rendimiento: 689.60%, Market Cap: 1915249.92, Volumen: 4851190.28
3. Symbol: MAVUSDT, Rendimiento: 313.60%, Market Cap: 236192.39, Volumen: 1142129.55
4. Symbol: AKROUSDT, Rendimiento: 231.25%, Market Cap: 388999.88, Volumen: 33361910.95
5. Symbol: CYBERUSDT, Rendimiento: 208.33%, Market Cap: 405659825.83, Volumen: 91364825.64
6. Symbol: SEIUSDT, Rendimiento: 126.09%, Market Cap: 3023111.19, Volumen: 20892268.05
7. Symbol: BNBDOWNUSDT, Rendimiento: 101.19%, Market Cap: 4191.22, Volumen: 190077.86
8. Symbol: BLZUSDT, Rendimiento: 92.32%, Market Cap: 7175685.24, Volumen: 58481542.28
9. Symbol: MKRUSDT, Rendimiento: 73.79%, Market Cap: 9982900368.16, Volumen: 9295065.52
10. Symbol: BCHUSDT, Rendimiento: 64.23%, Market Cap: 2004103717.58, Volumen: 10885951.75


In [113]:
import requests
import datetime

# Obtén la lista de todos los pares de criptomonedas y sus detalles
base_url_exchange_info = 'https://api.binance.com/api/v3/exchangeInfo'
response_exchange_info = requests.get(base_url_exchange_info)
exchange_info = response_exchange_info.json()

# Filtra los pares de criptomonedas contra USDT
usdt_pairs = [pair['symbol'] for pair in exchange_info['symbols'] if pair['quoteAsset'] == 'USDT']

# Calcula la fecha de inicio y fin para un período de 6 meses
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=180)

# Formatea las fechas en milisegundos para la API de Binance
start_date_ms = int(start_date.timestamp()) * 1000
end_date_ms = int(end_date.timestamp()) * 1000

# Obtén los precios y volúmenes de las criptomonedas en un período de 6 meses
base_url_klines = 'https://api.binance.com/api/v3/klines'
klines_params = {
    'symbol': '',
    'interval': '1d',
    'startTime': start_date_ms,
    'endTime': end_date_ms
}

# Calcula el market cap para cada criptomoneda y crea una lista de tuplas (symbol, market cap)
market_cap_data = []
for symbol in usdt_pairs:
    klines_params['symbol'] = symbol
    response_klines = requests.get(base_url_klines, params=klines_params)
    klines_data = response_klines.json()
    if klines_data:
        start_price = float(klines_data[0][1])
        end_price = float(klines_data[-1][4])
        volume = sum([float(kline[5]) for kline in klines_data])
        market_cap = end_price * volume
        market_cap_data.append((symbol, market_cap, volume))

# Ordena la lista de tuplas según el market cap de mayor a menor
sorted_market_cap_data = sorted(market_cap_data, key=lambda x: x[1], reverse=True)

# Imprime las mejores criptomonedas según su market cap en un período de 6 meses
for i, (symbol, market_cap, volume) in enumerate(sorted_market_cap_data[:10], start=1):
    print(f"{i}. Symbol: {symbol}, Market Cap: {market_cap:.2f}, Volumen: {volume:.2f}")


1. Symbol: BTCUSDT, Market Cap: 432838902417.46, Volumen: 16742923.77
2. Symbol: ETHUSDT, Market Cap: 115453963008.01, Volumen: 70741682.55
3. Symbol: USDCUSDT, Market Cap: 51226300448.25, Volumen: 51205818121.00
4. Symbol: BUSDUSDT, Market Cap: 47333820847.96, Volumen: 47314894890.00
5. Symbol: XRPUSDT, Market Cap: 37400679218.90, Volumen: 73277192827.00
6. Symbol: TUSDUSDT, Market Cap: 23462797651.42, Volumen: 23469838603.00
7. Symbol: BNBUSDT, Market Cap: 16732156516.40, Volumen: 81819836.27
8. Symbol: SOLUSDT, Market Cap: 15413653889.71, Volumen: 751152723.67
9. Symbol: ARBUSDT, Market Cap: 14275803491.07, Volumen: 14677980147.10
10. Symbol: DOGEUSDT, Market Cap: 11554150480.65, Volumen: 187354475120.00


In [48]:
import ccxt
import csv
from datetime import datetime

# Crear una instancia del intercambio CCXT (Binance en este caso)
exchange = ccxt.binance()

# Símbolos de las criptomonedas para analizar
crypto_symbols = ['BTC/USDT', 'ETH/USDT', 'BNB/USDT', 'ADA/USDT', 'DOGE/USDT', 'DOT/USDT', 'XRP/USDT' ,'SOL/USDT', 'UNI/USDT']

# Obtener las fechas de inicio reales para cada criptomoneda
start_dates = {}
for symbol in crypto_symbols:
    data = exchange.fetch_ohlcv(symbol, '1d')
    if data:
        start_dates[symbol] = exchange.iso8601(data[0][0])
    else:
        print(f"No se encontraron datos para {symbol}")

def fetch_historical_data(symbol, start_date):
    since = exchange.parse8601(start_date)
    ohlcv = exchange.fetch_ohlcv(symbol, '1d', since=since)
    return ohlcv

def write_csv_file(symbol, data):
    csv_file_path = f'{symbol.replace("/", "")}_data.csv'
    with open(csv_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
        csv_writer.writeheader()
        for data_point in data:
            timestamp = data_point[0] / 1000
            date = datetime.utcfromtimestamp(timestamp).strftime('%d/%m/%Y')
            csv_writer.writerow({
                'Date': date,
                'Open': data_point[1],
                'High': data_point[2],
                'Low': data_point[3],
                'Close': data_point[4],
                'Volume': data_point[5]
            })

for symbol in crypto_symbols:
    if symbol in start_dates:
        print(f"Procesando {symbol} desde {start_dates[symbol]}...")
        historical_data = fetch_historical_data(symbol, start_dates[symbol])
        write_csv_file(symbol, historical_data)
    else:
        print(f"No se pudo determinar la fecha de inicio para {symbol}")

print("Archivos CSV generados con éxito.")


BadSymbol: binance does not have market symbol USDT/USD